In [1]:
import pandas as pd
import random

In [2]:
# Veut dire que c’est un passage par adresse des 3 variables IX, IY et IZ.
# 3 variables entières qu’il faut initialiser avant le 1er appel de la fonction.
# Veut dire que la fonction est typée, elle retourne un nombre réel.
def alea(IX, IY, IZ):
    # Déclaration d’une variable inter de type réel
    inter = 0.0
    # A chaque exécution de la fonction, les valeurs des 3 variables sont recalculées, d’où un passage par adresse.
    # % : division entière
    # // : reste de la division entière
    IX = 171 * (IX % 177) - 2 * (IX // 177)
    IY = 172 * (IY % 176) - 35 * (IY // 176)
    IZ = 170 * (IZ % 178) - 63 * (IZ // 178)
    # S’il y a un dépassement de capacité sur une variable, on ajoute un nombre très grand, pour la machine 16 bits,
    # pour ramener à une valeur positive. Ce qui marche sur une machine 16 bits marche sur une 32 ou 64 bits.
    if IX < 0:
        IX = IX + 30269
    if IY < 0:
        IY = IY + 30307
    if IZ < 0:
        IZ = IZ + 30323
    inter = ((IX / 30269) + (IY / 30307) + (IZ / 30323))
    return [round(inter - int(inter), 4),IX, IY, IZ]

In [3]:
IX = 100
IY = 100
IZ = 100

In [4]:
salaire = [(40000, 30000), (30000, 20000), (20000, 15000), (15000, 10000), (10000, 7500), (7500, 5000), (5000, 3000)]
frequenceSalaire = [5, 10, 10, 15, 20, 20, 20]
age = [(61, 53), (52, 41), (40, 31), (30, 21)]
frequenceAge = [30, 30, 20, 20]
ageEmbauche = [(21, 22), (23, 24), (25, 28), (29, 32), (33, 35), (36, 40)]
frequenceAgeEmbauche = [5, 20, 25, 25, 20, 5]
salaire_embauche=[(32000,24000),(24000,16000),(16000,12000),(12000,8000),(8000,6000),(6000,4000),(4000,3000)]
frequenceSalaireEmbauche=[5,5,10,20,20,20,20]
years = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33]

In [5]:
ensemble_data_frames = {}
for j in range(1, 41):
    data_frames = {}
    for i in range(23, 34) :
        data_frames['Employe_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire", "cotisation"])
        data_frames['Retraite_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire"])
        data_frames['newRecru_20'+str(i)] = pd.DataFrame(columns=["id", "age", "salaire", "cotisation"])
    ensemble_data_frames['Simulation_'+str(j)] = data_frames

In [6]:
data_indices = {}
for j in range(1, 41):
    data_indices['Simulation_'+str(j)] = pd.DataFrame(columns=["year", "TotEmp", "TotRet", "TotCotis", "TotPens", "Reserve", "NouvRet", "NouvRec"])

In [28]:
def cumuleFrequenceIntervale(frequences):
    cumules = []
    indexisation = 0
    for i in frequences:
        cumules.append([indexisation, i+indexisation])
        indexisation += i
    return cumules

In [27]:
def indexFromTable(cumules, aleaNumber):
    i = 0
    for intervale in cumules :
        if intervale[1] > aleaNumber*100 :
            return i
        i+=1
    return i-1

In [9]:
def cotisation(salaire):
    if salaire < 5000:
        return salaire*0.05
    elif salaire < 7000 :
        return salaire*0.06
    elif salaire < 100000 :
        return salaire*0.07
    else:
        return salaire*0.1

In [10]:
def createEmployeRetraite(simulation, year, IX, IY, IZ):
    dfEmploye = ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)]
    dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)]
    for index in range(11000):
        if index >=1:
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaire), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAge), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire[salaireIndex][1], salaire[salaireIndex][0], 100)
        if index <10000:
            a = random.randrange(age[ageIndex][1], age[ageIndex][0], 1)

            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
            dfEmploye = pd.concat([dfEmploye, new_row], ignore_index=True)
        else:
            a = random.randint(65, 85)
            new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
            dfRetraite = pd.concat([dfRetraite, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]["Employe_20"+str(year)] = dfEmploye
    ensemble_data_frames["Simulation_"+str(simulation)]["Retraite_20"+str(year)] = dfRetraite

In [11]:
def pensionnaireFive(dfRetrait) :
    pensionnaire65 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    for index, row in dfRetrait.iterrows():
        a = row["age"]
        s = row["salaire"]
        p = ((25*2)/100)*row["salaire"]
        new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
        pensionnaire65 = pd.concat([pensionnaire65, new_row], ignore_index=True)
    return pensionnaire65

In [12]:
def pensionnaireTwo(dfEmploye, dfRetraite):
    pensionnaire62 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    pensionnaire65 = pensionnaireFive(dfRetraite)
    pensionnaire62 = pd.concat([pensionnaire62, pensionnaire65], ignore_index=True)
    for index, row in dfEmploye.iterrows():
        if row["age"]>=62:
            a = row["age"]
            s = row["salaire"]
            p = ((25*2)/100)*row["salaire"]
            new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
            pensionnaire62 = pd.concat([pensionnaire62, new_row], ignore_index=True)
    return pensionnaire62

In [13]:
def newRecrutement(IX, IY, IZ):
    Nouveux_Recrus = pd.DataFrame(columns=["id", "age", "salaire"])
    a=alea(IX, IY, IZ)[0]
    n = ((400 - 250) * a) + 250
    for index in range(int(n)):
        if index >= 0 :
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaireEmbauche), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAgeEmbauche), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire_embauche[salaireIndex][1], salaire_embauche[salaireIndex][0], 100)
        a = random.randrange(ageEmbauche[ageIndex][0], ageEmbauche[ageIndex][1], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
        Nouveux_Recrus = pd.concat([Nouveux_Recrus, new_row], ignore_index=True)
    return Nouveux_Recrus

In [14]:
def calculEmployeRetraite(simulation, i, IX, IY, IZ):
    EmployeBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i-1)]
    RetraiteBeta = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i-1)]
    EmployeAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)]
    RetraiteAlpha = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)]
    for index, row in EmployeBeta.iterrows():
        if row["age"] < 64:
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s], "cotisation" : [cotisation(s)]})
            EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
        else :
            if i == 25 or i == 30:
                s = row["salaire"]*1.05
            else :
                s = row["salaire"]
            a = row["age"]+1
            new_row = pd.DataFrame({"id" : row["id"], "age": [a], "salaire": [s]})
            RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    for index, row in RetraiteBeta.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]+1], "salaire": [row["salaire"]]})
        RetraiteAlpha = pd.concat([RetraiteAlpha, new_row], ignore_index=True)
    newRecru = newRecrutement(IX, IY, IZ)
    for index, row in newRecru.iterrows():
        new_row = pd.DataFrame({"id" : [row["id"]], "age": [row["age"]], "salaire": [row["salaire"]],"cotisation" : [row["cotisation"]]})
        EmployeAlpha = pd.concat([EmployeAlpha, new_row], ignore_index=True)
    ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(i)] = EmployeAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(i)] = RetraiteAlpha
    ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(i)] = newRecru

In [15]:
def calculIndice(simulation, ageRetrait):
    dfIndice = data_indices["Simulation_"+str(simulation)]
    dfEmploye =  ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(23)]
    dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(23)]
    TotEmp = dfEmploye.shape[0]
    if ageRetrait == 62:
        TotCotis = dfEmploye[dfEmploye["age"]<=61]["cotisation"].sum()*12
        pensionnaire = pensionnaireTwo(dfEmploye, dfRetraite)
    else:
         TotCotis = dfEmploye["cotisation"].sum()*12
         pensionnaire = pensionnaireFive(dfRetraite)
    TotRet = pensionnaire.shape[0]
    TotPens = pensionnaire["pension"].sum()*12
    Reserve = 200000000 + TotCotis - TotPens
    NouvRet = 0
    NouvRec = 0
    new_row = pd.DataFrame({"year" : [2023], "TotEmp" : [TotEmp], "TotRet" : [TotRet], "TotCotis" : [TotCotis], "TotPens" : [TotPens], "Reserve" : [Reserve], "NouvRet" : [NouvRet], "NouvRec" : [NouvRec]})
    dfIndice = pd.concat([dfIndice, new_row], ignore_index=True)
    for year in years :
        dfEmploye =  ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)]
        dfRetraite = ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)]
        dfRecrutement = ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(year)]
        TotEmp = dfEmploye.shape[0]
        if ageRetrait == 62:
            TotCotis = dfEmploye[dfEmploye["age"]<=61]["cotisation"].sum()*12
            pensionnaire = pensionnaireTwo(dfEmploye, dfRetraite)
        else:
            pensionnaire = pensionnaireFive(dfRetraite)
            TotCotis = dfEmploye["cotisation"].sum()*12
        TotPens = pensionnaire["pension"].sum()*12
        TotRet = pensionnaire.shape[0]
        Reserve = dfIndice.iloc[year-24]['Reserve'] + TotCotis -TotPens
        NouvRet = TotRet - dfIndice.iloc[year-24]['TotRet']
        NouvRec =  dfRecrutement.shape[0]
        new_row = pd.DataFrame({"year" : [2000+year], "TotEmp" : [TotEmp], "TotRet" : [TotRet], "TotCotis" : [TotCotis], "TotPens" : [TotPens], "Reserve" : [Reserve], "NouvRet" : [NouvRet], "NouvRec" : [NouvRec]})
        dfIndice = pd.concat([dfIndice, new_row], ignore_index=True)
    return dfIndice

In [16]:
createEmployeRetraite(1, 23, IX, IY, IZ)
for year in years:
    calculEmployeRetraite(1, year, IX, IY, IZ)

In [17]:
simulation = 1
year = 33
ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)]

,id,age,salaire,cotisation
0,1,47,8048.25,563.3775
1,2,56,10253.25,717.7275
2,4,32,4520.25,226.0125
3,5,35,3969.00,198.4500
4,6,39,3528.00,176.4000
...,...,...,...,...
11289,349,30,7900.00,553.0000
11290,350,30,4900.00,245.0000
11291,351,34,5500.00,330.0000
11292,352,23,19200.00,1344.0000


In [18]:
ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)]

,id,age,salaire
0,24,65,19734.75
1,47,65,41454.00
2,56,65,20065.50
3,68,65,23703.75
4,86,65,41013.00
...,...,...,...
3231,10996,94,7700.00
3232,10997,77,37600.00
3233,10998,75,3300.00
3234,10999,94,9900.00


In [19]:
pensionnaireTwo(ensemble_data_frames["Simulation_"+str(simulation)]['Employe_20'+str(year)], ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)])

,id,age,salaire,pension
0,24,65,19734.75,9867.375
1,47,65,41454.00,20727.000
2,56,65,20065.50,10032.750
3,68,65,23703.75,11851.875
4,86,65,41013.00,20506.500
...,...,...,...,...
3979,9905,64,20947.50,10473.750
3980,9931,63,33516.00,16758.000
3981,9943,63,38256.75,19128.375
3982,9983,64,16206.75,8103.375


In [20]:
pensionnaireFive(ensemble_data_frames["Simulation_"+str(simulation)]['Retraite_20'+str(year)])

,id,age,salaire,pension
0,24,65,19734.75,9867.375
1,47,65,41454.00,20727.000
2,56,65,20065.50,10032.750
3,68,65,23703.75,11851.875
4,86,65,41013.00,20506.500
...,...,...,...,...
3231,10996,94,7700.00,3850.000
3232,10997,77,37600.00,18800.000
3233,10998,75,3300.00,1650.000
3234,10999,94,9900.00,4950.000


In [21]:
ensemble_data_frames["Simulation_"+str(simulation)]['newRecru_20'+str(year)]

,id,age,salaire,cotisation
0,1,29,7500,525.0
1,2,23,15500,1085.0
2,3,36,3500,175.0
3,4,33,3300,165.0
4,5,34,3900,195.0
...,...,...,...,...
348,349,30,7900,553.0
349,350,30,4900,245.0
350,351,34,5500,330.0
351,352,23,19200,1344.0


In [22]:
calculIndice(1, 62)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,94668876.00,68010600.0,2.266583e+08,0,0
1,2024,10353,1000,97078932.00,68010600.0,2.557266e+08,0,353
2,2025,10706,1373,97588797.60,118457220.0,2.348582e+08,373,353
3,2026,11059,1751,92657740.20,170739030.0,1.567769e+08,378,353
4,2027,11412,2118,87839047.20,222074580.0,2.254136e+07,367,353
5,2028,11392,2502,82687836.00,276000060.0,-1.707709e+08,384,353
6,2029,11367,2877,77889035.40,327409950.0,-4.202918e+08,375,353
7,2030,11353,3236,77156801.25,388473078.0,-7.316081e+08,359,353
8,2031,11322,3621,71775540.27,443918685.0,-1.103751e+09,385,353
9,2032,11300,3984,66507657.99,498788787.0,-1.536032e+09,363,353


In [23]:
calculIndice(1, 65)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,9.466888e+07,68010600.0,2.266583e+08,0,0
1,2024,10353,1000,9.707893e+07,68010600.0,2.557266e+08,0,353
2,2025,10706,1000,1.046513e+08,68010600.0,2.923673e+08,0,353
3,2026,11059,1000,1.070397e+08,68010600.0,3.313965e+08,0,353
4,2027,11412,1000,1.094080e+08,68010600.0,3.727939e+08,0,353
5,2028,11392,1373,1.047438e+08,118457220.0,3.590805e+08,373,353
6,2029,11367,1751,9.982296e+07,170739030.0,2.881644e+08,378,353
7,2030,11353,2118,1.000932e+08,224641357.5,1.636163e+08,367,353
8,2031,11322,2502,9.454732e+07,281263111.5,-2.309950e+07,384,353
9,2032,11300,2877,8.940400e+07,335243496.0,-2.689390e+08,375,353


In [24]:
ListSimulation = []
for i in range (40):
    if i == 0 :
        ListSimulation.append({"IX": 100, "IY" : 100, "IZ": 100})
    else :
        ListSimulation.append({"IX": ListSimulation[i-1]["IX"] + 10, "IY": ListSimulation[i-1]["IY"] + 20, "IZ": ListSimulation[i-1]["IZ"] + 30})

In [25]:
def createSimulation(Simulation, X, Y, Z) :
    createEmployeRetraite(Simulation, 23, X, Y, Z)
    for year in years:
        calculEmployeRetraite(Simulation, year, X, Y, Z)

In [29]:
createSimulation(2, ListSimulation[1]["IX"], ListSimulation[1]["IY"], ListSimulation[1]["IZ"])

In [30]:
for i in range(3, 41):
    createSimulation(i, ListSimulation[i-1]["IX"], ListSimulation[i-1]["IY"], ListSimulation[i-1]["IZ"])

KeyboardInterrupt: 

In [37]:
ensemble_data_frames["Simulation_22"]['Retraite_2032']

,id,age,salaire
0,6,65,19183.50
1,62,65,15435.00
2,63,65,17088.75
3,81,65,34949.25
4,84,65,14553.00
...,...,...,...
2869,10996,78,4800.00
2870,10997,74,4900.00
2871,10998,75,17800.00
2872,10999,75,3600.00


In [38]:
calculIndice(21, 62)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,94995000.00,69483000.0,2.255120e+08,0,0
1,2024,10318,1000,97274292.00,69483000.0,2.533033e+08,0,318
2,2025,10636,1393,97016248.20,125042700.0,2.252768e+08,393,318
3,2026,10954,1738,92599737.60,173031690.0,1.448449e+08,345,318
4,2027,11272,2127,87260181.60,227614890.0,4.490179e+06,389,318
5,2028,11197,2507,82263364.80,279592410.0,-1.928389e+08,380,318
6,2029,11170,2883,77052018.60,333149340.0,-4.489362e+08,376,318
7,2030,11099,3252,76037937.90,395237478.0,-7.681357e+08,369,318
8,2031,11037,3616,70995169.35,447598510.5,-1.144739e+09,364,318
9,2032,10979,4012,65232130.68,505180101.0,-1.584687e+09,396,318


In [39]:
calculIndice(21, 65)

,year,TotEmp,TotRet,TotCotis,TotPens,Reserve,NouvRet,NouvRec
0,2023,10000,1000,9.499500e+07,69483000.0,2.255120e+08,0,0
1,2024,10318,1000,9.727429e+07,69483000.0,2.533033e+08,0,318
2,2025,10636,1000,1.047946e+08,69483000.0,2.886149e+08,0,318
3,2026,10954,1000,1.070966e+08,69483000.0,3.262285e+08,0,318
4,2027,11272,1000,1.093986e+08,69483000.0,3.661441e+08,0,318
5,2028,11197,1393,1.039003e+08,125042700.0,3.450017e+08,393,318
6,2029,11170,1738,9.946849e+07,173031690.0,2.714385e+08,345,318
7,2030,11099,2127,9.912302e+07,230344050.0,1.402175e+08,389,318
8,2031,11037,2507,9.377010e+07,284920446.0,-5.093286e+07,380,318
9,2032,10979,2883,8.819561e+07,341155222.5,-3.038925e+08,376,318
